In [ ]:
import hilbert
import conversions

#includes for genetic
import population
import psnr
import crossover


In [ ]:
x, y = hilbert.genHilbert(512, 512)

In [ ]:
def genPoint(offsetX,offsetY):
    for i in range(512*512):
        x[i]=(x[i]+offsetX)%512
    for i in range(512*512):
        y[i]=(y[i]+offsetY)%512
    return x,y

In [ ]:
import cv2

# image_name = input("Enter Cover Image Name: ")
image_name = "lena_gray.bmp"
image = cv2.imread(image_name, 0)


Creating Secret Data from the Secret Image

In [ ]:
secret_image = "lena_secret.bmp"
secretImage = cv2.imread(image_name, 0)

b = bytearray(secretImage)

secretData = ""

for i in b:
    secretData = secretData + str(conversions.int2binary(i))

secretFile = open("secret.txt", 'w')
secretFile.write(secretData)



In [ ]:

def encodeImage(image,x,y):
    secretDataIndex=0
    for i in range(512*512):
        r = conversions.int2binary(image[x[i]][y[i]])
        image[x[i]][y[i]] = int(
            r[:-3] + str(secretData[secretDataIndex]) + str(secretData[secretDataIndex+1]) + str(secretData[secretDataIndex+2]), 2)
        secretDataIndex = secretDataIndex+3
    file_name = "encoded_image.bmp"
    cv2.imwrite(file_name, image)

    return cv2.imread(file_name, 0)

In [ ]:
def encodeFunction(strr, image):
    sbDir = strr[0]
    sbPole = strr[1]
    yValue = strr[2:11]
    xValue = strr[11:20]

    xOff = int(xValue,2)
    yOff = int(yValue,2)

    x,y=genPoint(xOff,yOff)
    stegoImage = encodeImage(image, x,y)

    return stegoImage

Generating Initial Population For Genetic

In [ ]:
it = 4

mainPopulation = population.genPopulation(4)

print(mainPopulation)
fitnessArray = []*it


for i in range(it):
    print("Parent - ", i)
    image = cv2.imread(image_name, 0)
    stegoImage = encodeFunction(mainPopulation[i],image)
    originalImage = cv2.imread(image_name, 0)
    fitnessArray.append(psnr.computeFitness(512, originalImage, stegoImage))

    print(fitnessArray)

